In [1]:
import pandas as pd
import sqlite3

dataframes needed:
- one with hospitals (Michael said there wasn't any real programatic way to do it, get a df with entity_type = 2 and then figure out the real list by looking at names)
- use hospital df to get list of referees

In [32]:
# building table for hospitals
# cbsa = 34980, either hardcode or use a cte to get the value where usps_zip_pref_city = "NASHVILLE" and usps_zip_pref_state = "TN" (filter based on referee cbsa)
query = """
SELECT npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE entity_type_code = 2
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp_sqlite = pd.read_sql(query, db)

hosp_sqlite.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1881697092,2,"RHS, INC.",1330 TROTWOOD AVE,None,COLUMBIA,TN,38401,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
1,1326041534,2,MURFREESBORO CONVALESCENT SERVICE,4428 LASCASSAS PIKE,None,MURFREESBORO,TN,37130,341600000X,Transportation Services,Ambulance,Ambulance
2,1063415834,2,ADVANCED DIABETIC SERVICES,151 HERITAGE PARK DR,STE 403,MURFREESBORO,TN,37129,332B00000X,Suppliers,Durable Medical Equipment & Medical Supplies,Durable Medical Equipment & Medical Supplies
3,1538162334,2,TENNESSEE IMAGING ALLIANCE LLC,2628 N MOUNT JULIET RD,None,MOUNT JULIET,TN,37122,261QR0208X,Ambulatory Health Care Facilities,Clinic/Center,Mobile Radiology Clinic/Center
4,1245233220,2,"NORTHRIDGE SURGERY CENTER, LP",601 SAUNDERSVILLE RD,None,MADISON,TN,37075,261QA1903X,Ambulatory Health Care Facilities,Clinic/Center,Ambulatory Surgical Clinic/Center


In [34]:
hosp_sqlite['code_group'].value_counts()

Allopathic & Osteopathic Physicians                                             2002
Ambulatory Health Care Facilities                                               1408
Suppliers                                                                       1354
Agencies                                                                         888
Dental Providers                                                                 662
Behavioral Health & Social Service Providers                                     440
Nursing & Custodial Care Facilities                                              332
Chiropractic Providers                                                           332
Physician Assistants & Advanced Practice Nursing Providers                       329
Transportation Services                                                          327
Other Service Providers                                                          317
Respiratory, Developmental, Rehabilitative and Restorative Servic

In [58]:
# list of "hospital" NPIs to build profile
# all located in Nashville CBSA
hosp_npi = tuple(hosp_sqlite[hosp_sqlite['code_group'].isin(['Hospitals', 'Hospital Units', 'Ambulatory Health Care Facilities'])]['npi'].tolist())

In [103]:
# get dataframe of hospitals in Nashville CBSA using previously generated NPI list
query = f"""
SELECT CAST(npi AS int64) AS npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE npi IN {hosp_npi}
    AND cbsa = 34980;
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hosp = pd.read_sql(query, db)

hosp.head()

,npi,entity_type,provider_org_name,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1003107673,2,"MEHARRY MEDICAL COLLEGE, SCHOOL OF DENTISTRY",1007 DB TODD BLVD,"MEHARRY MEDICAL COLLEGE, SCHOOL OF DENTISTRY 1...",NASHVILLE,TN,37208,261QD0000X,Ambulatory Health Care Facilities,Clinic/Center,Dental Clinic/Center
1,1003211145,2,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,2200 CHILDRENS WAY PRESTON RESEARCH BLDG 395,None,NASHVILLE,TN,37232,282NC2000X,Hospitals,General Acute Care Hospital,Children's Hospital
2,1003342247,2,"GUARDIAN HEALTHCARE PROVIDERS, INC.",105 WESTPARK DR,SUITE 100,BRENTWOOD,TN,37027,261QP2000X,Ambulatory Health Care Facilities,Clinic/Center,Physical Therapy Clinic/Center
3,1003362294,2,TOTAL RENAL CARE INC,5209 LINBAR DR STE 605,None,NASHVILLE,TN,37211,261QE0700X,Ambulatory Health Care Facilities,Clinic/Center,End-Stage Renal Disease (ESRD) Treatment Clini...
4,1003379108,2,"INFUSION EXPRESS OF TENNESSEE, PC",1006 GLENBROOK WAY,STE 130,HENDERSONVILLE,TN,37075,261Q00000X,Ambulatory Health Care Facilities,Clinic/Center,Clinic/Center


In [104]:
# query to get to_npi from hop data where to_npi is in hospital list
query = f"""
SELECT from_npi,
    to_npi,
    patient_count,
    transaction_count,
    average_day_wait,
    std_day_wait
FROM hop
WHERE to_npi IN {hosp_npi};
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    hop_to_hosp = pd.read_sql(query, db)

hop_to_hosp.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1427251859,1003362294,29,74,0.770,4.403
1,1700878360,1003362294,18,61,21.328,12.353
2,1093741464,1003362294,20,57,19.070,43.333
3,1255335535,1003362294,28,55,15.745,17.589
4,1518346469,1013115005,90,161,23.006,27.651


to construct dataframe for provider profile:
- query to get rest of provider info (from_npi in hop_to_hosp)
- inner join on npi = from_npi in hop_to_hosp
- join hop_to_hosp with hosp on npi = to_npi to get hospital names (think I can drop most columns from hospital, fine to go back and keep some later if needed)

In [105]:
hop_to_hosp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18486 entries, 0 to 18485
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   from_npi           18486 non-null  int64  
 1   to_npi             18486 non-null  int64  
 2   patient_count      18486 non-null  int64  
 3   transaction_count  18486 non-null  int64  
 4   average_day_wait   18486 non-null  float64
 5   std_day_wait       18486 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 866.7 KB


In [106]:
# tuple of providers who referred to area hospitals
provider_npi = tuple(hop_to_hosp['from_npi'].unique().tolist())

In [107]:
# get dataframe of hospitals in Nashville CBSA using previously generated NPI list
query = f"""
SELECT CAST(npi AS int64) AS npi,
    entity_type_code AS entity_type,
    provider_organization_name_legal_business_name AS provider_org_name,
    provider_last_name_legal_name AS provider_last_name,
    provider_first_name,
    provider_middle_name,
    provider_name_prefix_text AS provider_name_prefix,
    provider_name_suffix_text AS provider_name_suffix,
    provider_credential_text AS provider_credential,
    provider_first_line_business_practice_location_address AS provider_address_1,
    provider_second_line_business_practice_location_address AS provider_address_2,
    provider_business_practice_location_address_city_name AS provider_city,
    provider_business_practice_location_address_state_name AS provider_state,
    provider_business_practice_location_address_postal_code AS provider_post_code,
    extracted_taxonomy_code AS tax_code,
    grouping AS code_group,
    classification AS code_classification,
    display_name AS code_display_name
FROM npi AS n
LEFT JOIN class_cross AS c
    ON n.extracted_taxonomy_code = c.code
LEFT JOIN cbsa_cross cb
    ON n.provider_business_practice_location_address_postal_code = cb.zip
WHERE npi IN {provider_npi};
"""

with sqlite3.connect('../data/provider.sqlite') as db: 
    provider = pd.read_sql(query, db)

provider.head()

,npi,entity_type,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_address_1,provider_address_2,provider_city,provider_state,provider_post_code,tax_code,code_group,code_classification,code_display_name
0,1003013160,1,None,GRABENSTEIN,WILLIAM,P.,DR.,None,M.D.,1822 MEMORIAL DR,None,CLARKSVILLE,TN,37043,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,Family Medicine Physician
1,1003013160,1,None,GRABENSTEIN,WILLIAM,P.,DR.,None,M.D.,1822 MEMORIAL DR,None,CLARKSVILLE,TN,37043,207Q00000X,Allopathic & Osteopathic Physicians,Family Medicine,Family Medicine Physician
2,1003013947,1,None,OTTO,ROBERT,JOEL,DR.,None,M.D.,2400 PATTERSON ST STE 100,None,NASHVILLE,TN,37203,207X00000X,Allopathic & Osteopathic Physicians,Orthopaedic Surgery,Orthopaedic Surgery Physician
3,1003019902,1,None,ACOSTA,LEALANI,MAE,None,None,MD,3601 TVC,None,NASHVILLE,TN,37232,2084N0400X,Allopathic & Osteopathic Physicians,Psychiatry & Neurology,Neurology Physician
4,1003028770,2,SPRING HILL DERMATOLOGY PLC,None,None,None,None,None,None,1229 RESERVE BLVD,SUITE 200,SPRING HILL,TN,37174,207N00000X,Allopathic & Osteopathic Physicians,Dermatology,Dermatology Physician


In [108]:
provider.info()
# shorter dataframe than length of hop_to_hosp, some providers referred sufficient patients to multiple facilities to be duplicated in hop data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10266 entries, 0 to 10265
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   npi                   10266 non-null  int64 
 1   entity_type           10180 non-null  object
 2   provider_org_name     2696 non-null   object
 3   provider_last_name    7482 non-null   object
 4   provider_first_name   7484 non-null   object
 5   provider_middle_name  5649 non-null   object
 6   provider_name_prefix  3029 non-null   object
 7   provider_name_suffix  335 non-null    object
 8   provider_credential   7219 non-null   object
 9   provider_address_1    10180 non-null  object
 10  provider_address_2    2772 non-null   object
 11  provider_city         10180 non-null  object
 12  provider_state        10180 non-null  object
 13  provider_post_code    10180 non-null  object
 14  tax_code              10156 non-null  object
 15  code_group            10156 non-null

In [109]:
# results in ~4000 more rows than hop_to_hosp, figure it out later
referral = pd.merge(provider, hop_to_hosp, left_on = 'npi', right_on = 'from_npi', how = 'inner').drop(columns = 'from_npi')

In [114]:
# add hospital name and code_class
referral = pd.merge(referral, hosp[['npi', 'provider_org_name', 'code_classification']], how = 'inner', left_on = 'to_npi', right_on = 'npi').drop(columns = 'npi_y').rename({'npi_x': 'provider_npi', 'provider_org_name_x': 'provider_org_name', 'provider_org_name_y': 'hospital_name', 'code_classification_y': 'hospital_code_class'}, axis = 1)

In [116]:
referral.to_csv('../data/referral.csv', index = False)